# Deploying pointnet height models 

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [ ]:
from azureml.core.model import Model

### Registering the model to the workspace

In [ ]:
#skip if model is already registered
model = Model.register(model_path = "./pointnet/",
                       model_name = "pointnet-height-20190806",
                       description = "pointnet model trained for predicting height",
                       workspace = ws)

### If the model is already registered in the workspace fetch it using the model name

In [ ]:
model = Model(ws, name='pointnet-height-20190806')

In [ ]:
model

### Entry script for deploying on a AKS(Azure Kubernetes Service)

In [ ]:
%%writefile score_aks.py
import json
import numpy as np
import os
import sys

sys.path.append('/structure/azureml-app/azureml-models/pointnet-height-20190806/1/pointnet')
#sys.path.append('/var/azureml-app/azureml-models/20190806-1551_220-55height/1/20190806-1551_220-55height')
import modelutils, utils

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    input_shape = [10000, 3]
    output_size = 1
    hidden_sizes = [512, 256, 128]
    weights_path = '/structure/azureml-app/azureml-models/pointnet-height-20190806/1/pointnet/20190806-1551_220-55height-pointnet-model-weights.h5'
    model = modelutils.load_pointnet(weights_path, input_shape, output_size, hidden_sizes)
    
def run(data):
    try:
        data_list = json.loads(data)
        data_np = np.array(data_list['data'])
        result = model.predict(data_np)
        # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

### Entry script to deploy to ACI(Azure Container Instance)

In [ ]:
%%writefile score_aci.py
import json
import numpy as np
import os
import sys

sys.path.append('/var/azureml-app/azureml-models/pointnet-height-20190806/1/pointnet')
import modelutils, utils

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    
    input_shape = [10000, 3]
    output_size = 1
    hidden_sizes = [512, 256, 128]
    weights_path = '/var/azureml-app/azureml-models/pointnet-height-20190806/1/pointnet/20190806-1551_220-55height-pointnet-model-weights.h5' 
    model = modelutils.load_pointnet(weights_path, input_shape, output_size, hidden_sizes)
    
def run(data):
    try:
        data_list = json.loads(data)
        data_np = np.array(data_list['data'])
        result = model.predict(data_np)
        # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

### Define the dependencies required for the model and set inference config

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment('pointnet_height')
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow'],pip_packages=['azureml-defaults', 'pyntcloud', 'inference_schema', 'opencv-python', 'matplotlib', 'psycopg2-binary', 'tqdm', 'Pillow', 'opencv-python', 'bunch'])
inference_config_aci = InferenceConfig(entry_script="score_aci.py", environment=env)
inference_config_aks = InferenceConfig(entry_script="score_aks.py", environment=env)

### Use to register the environment to the workspace

In [ ]:
env.register(ws)

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

### Specify the deployment config and deploy the service

In [ ]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4)
service = Model.deploy(ws, "aci-pointnet-height-20190806", [model], inference_config_aci, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

### check the service logs if things don't work

In [ ]:
service.get_logs()

## Creating Inference cluster

For provisioning_configuration(), if you pick custom values for agent_count and vm_size, and cluster_purpose is not DEV_TEST, then you need to make sure agent_count multiplied by vm_size is greater than or equal to 12 virtual CPUs. For example, if you use a vm_size of "Standard_D3_v2", which has 4 virtual CPUs, then you should pick an agent_count of 3 or greater.

### skip if already created

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this).
# For example, to create a dev/test cluster, use:
# prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
#prov_config = AksCompute.provisioning_configuration()

aks_name = 'pointnet-height'

prov_config = prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_A2_v2",
                                                       agent_count = 6,
                                                       location = "westeurope")

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice

### Deploy the model on created AKS

In [ ]:
aks_target = AksCompute(ws,"pointnet-height")

deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4)
service = Model.deploy(ws, "aks-pointnet-height-20190806", [model], inference_config_aks, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)

In [ ]:
service.get_logs()